# Tratamento de exceções em Python

## 1 - Um anti-exemplo, ou por que usar exceções?

Suponha o seguinte cenário:
Deseja-se obter o número da versão linux instalada na máquina virtual do colab.

Uma maneira é abrir o arquivo /etc/issue. Em distribuições linux, este arquivo contém uma cadeia de caracteres com a versão atual do sistema operacional.


In [ ]:
print(open("/etc/issue").read())

Ubuntu 22.04.3 LTS \n \l




Infelizmente, este método é sujeito a erros.
Em primeiro lugar, o arquivo pode não existir, pode não ser legível e os dados nele podem não estarem em um formato reconhecível.
Estes erros devem ser identificados e tratados.

Na forma "clássica" (sem exceções), as funções que lidam com este tipo de incerteza devem retornar um par de valores, um código de erro e o valor buscado.

Por exemplo, a função a seguir retorna uma tupla cujo primeiro elemento é uma mensagem de erro ou o valor ```None``` caso tenha sido bem sucedida.



In [ ]:
import pathlib
import os
import re

reg = re.compile("Ubuntu\\s+([0-9]+)(\\.([0-9]+))?(\\.([0-9]+))?")
def recupera_versao():
  caminho = "/etc/issue"
  if pathlib.Path(caminho).is_file():
    if os.access(caminho, os.R_OK):
      p = open(caminho)
      s = p.read()
      if len(s)>0:
        match = reg.match(s)
        if match and len(match.groups())>1:
          versoes = (int(match.groups()[0]),)
          for i in range(2, len(match.groups()),2):
            versoes = versoes + (int(match.groups()[i]),)
          ret = (None,) + versoes
        else:
          ret = ("Erro: Conteúdo do arquivo não pôde ser interpretado", None)
      else:
        ret = ("Erro: Arquivo não contém nada", None)
    else:
      ret = ("Erro: Arquivo não pode ser lido", None)

  else:
    ret = ("Erro: Arquivo não existe", None)

  return ret

In [ ]:
recupera_versao()

(None, 22, 4, 3)

Este tipo de código é conhecido por programadores como *the pyramid of doom* (a pirâmide da desgraça).
Veja o formato triangular que a edentação do código toma.
Note também como boa parte do código se preocupa com os ditos "fluxos alternativos", ou seja, caminhos adotados pelo código quando ocorre um erro.

Isso fica mais grave quando estas funções são chamadas dentro de outras funções.

Por exemplo, suponha que deseja-se carregar alguma biblioteca na versão 2 se a versão Ubuntu estiver entre 16 e 18 (não-incluso) e versão 3 se for 18 ou maior. Versões menores que 16 não são suportadas.

O código seguiria este formato:

In [ ]:
def carrega_lib(versao):
  print("Carregando bibliotecas da versão " + str(versao))
  # carrega bibliotecas de uma determinada versão
  return None

def carrega_bibliotecas():
  versao = recupera_versao()
  if versao[0] is None:
    if versao[1]>=18:
      err = carrega_lib(3)
      if err is None:
        ret = None
      else:
        ret = "Erro ao carregar a versão. O erro reportado foi: " + err
      ret = None
    elif versao[1]>=16 and versao[1]<18:
      ret = carrega_lib(2)
      ret = None
      if err is None:
        ret = None
      else:
        ret = "Erro ao carregar a versão. O erro reportado foi: " + err
    else:
      ret = "Versão não suportada"
  else:
    ret = "Erro ao obter a versão do sistema operacional. O Erro reportado foi:" + versao[0]
  return ret


In [ ]:
carrega_bibliotecas()

Carregando bibliotecas da versão 3


Novamente, apresenta-se o mesmo problema da "pyramid of doom". Novamente, a maior parte do código se preocupa com fluxos alternativos.

Este é o modo de se tratar situações de erro em vários ambientes, em particular na linguagem C.

Python e outras linguagens no entanto nos fornecem uma alternativa, as exceções.

## 2 - Exceções: uma alternativa mais elegante

Exceções são *desvios informativos não-locais do fluxo de execução*.
O que isso significa?
Uma exceção é uma maneira de se desviar o fluxo de execução para fora do escopo da função que está sendo executada.
Este desvio carrega consigo alguma informação, tradicionalmente, um código ou uma mensagem de erro.
Elas são empregadas para sinalizar situações excepcionais como erros.
A não-localidade significa que as exceções "atravessam" chamadas de funções e podem ser encaminhadas diretamente ao código que pode tratá-las.  

Por exemplo, a função anterior pode ser reimplementada da seguinte forma:


In [ ]:
import pathlib
import os
import re

reg = re.compile("Ubuntu\\s+([0-9]+)(\\.([0-9]+))?(\\.([0-9]+))?")
def recupera_versao():
  caminho = "/etc/issue"
  p = open(caminho)
  s = p.read()
  match = reg.match(s)
  versoes = (int(match.groups()[0]),)
  for i in range(2, len(match.groups()),2):
    versoes = versoes + (int(match.groups()[i]),)

  return versoes

In [ ]:
p = open("/etc/issue")

In [ ]:
p.close()

Note que o código que verifica cada erro potencial foi removido.
O que acontece no caso do arquivo não existir, não ser legível, não poder ser interpretado?

As falhas em cada um destes pontos gerariam, pela própria *runtime* do Python (sistemas de arquivos, expressões regulares e afins), exceções.

Você mesmo já deve ter se deparado com uma exceção.

Por exemplo, o código a seguir gera uma exceção ```IndexError``` ao tentar acessar uma posição inválida da sequência.

In [ ]:
[1,2,3][3]

IndexError: list index out of range

In [ ]:
[None, 2, 3][0]

As exceções atravessam todas as chamadas de função (interrompendo imediatamente a sua execução) até encontrar algum escopo apto a tratá-la.
Se não há tratamento, a execução do código encerra-se (como no exemplo acima).

Exceções são tratadas em blocos do tipo ```try``` ... ```except``` ...

Este é um exemplo

In [ ]:
valores = ['a', 'b', 'c', 'd']

def mostra_valores():
  try:
    for i in range(20):
      print(str(i)+": " + valores[i])
  except:
    print("Fui longe demais!")

mostra_valores()

0: a
1: b
2: c
3: d
Fui longe demais!


Observe que o fluxo do código dentro do bloco ```try``` é desviado para o bloco ```except``` assim que ocorre uma exceção.

Este desvio atravessa até mesmo chamadas de função!

In [ ]:
valores = ['a', 'b', 'c', 'd']

def recupera_valor(indice):
  return valores[indice]

def mostra_valores():
  try:
    for i in range(20):
      print(str(i)+": " + recupera_valor(i))
  except:
    print("Fui longe demais!")

mostra_valores()

0: a
1: b
2: c
3: d
Fui longe demais!


Note como a exceção ocorre dentro da função ```recupera_valores```, no entanto, o código é desviado para o bloco ```except``` dentro da função ```mostra_valores```.

Vários blocos except são possíveis para tratar exceções de diferentes tipos.

Por exemplo:

In [ ]:
valores = ['1', '2', '3.5', '4']

def recupera_valor(indice):
    return int(valores[indice])

def mostra_valores():
  for i in range(10):
    try:
      print(str(i)+": " + str(recupera_valor(i)))
    except (ValueError):
      print(str(i)+": não é inteiro")

mostra_valores()

0: 1
1: 2
2: não é inteiro ou índice inválido
3: 4
4: não é inteiro ou índice inválido
5: não é inteiro ou índice inválido
6: não é inteiro ou índice inválido
7: não é inteiro ou índice inválido
8: não é inteiro ou índice inválido
9: não é inteiro ou índice inválido


In [ ]:
try:
  raise Exception("erro")
except Exception as err:
  print(err)

erro


O bloco adequado é selecionado de acordo com a classe do objeto exceção que foi lançado.

Objetos exceção são lançados com o comando ```raise```.

Assim, a função ```carrega_bibliotecas()``` pode sinalizar um erro de versão não-suportada.

In [ ]:
def carrega_lib(versao):
  print("Carregando bibliotecas da versão " + str(versao))
  # carrega bibliotecas de uma determinada versão

def carrega_bibliotecas():
  versao = recupera_versao()
  if versao[0]>=18:
    carrega_lib(3)
  elif versao[0]>=16 and versao[1]<18:
    carrega_lib(2)
  else:
    raise RuntimeError("Versão não suportada")

O erro é reportado pelo lançamento de uma exceção do tipo "RuntimeError". Note que as demais exceções eventualmente lançadas dentro de ```recupera_versao``` "atravessam" a função ```carrega_bibliotecas``` e também são reportadas.

In [ ]:
def recupera_versao():
  return [14]

carrega_bibliotecas()

RuntimeError: Versão não suportada

Programadores também podem criar os seus próprios tipos de exceção.
Isso é bastante útil, pois como dito, o bloco ```except``` é selecionado de acordo com o tipo da exceção.

Para tanto, os programadores devem criar as suas próprias exceções, *sempre* derivadas da classe ```Exception``` ou de alguma de suas filhas.

Eis um novo exemplo:



In [ ]:
import pathlib
import os
import re

class ErroVersao(Exception):
  pass

caminho = "/etc/issue"
reg = re.compile("Ubuntu\\s+([0-9]+)(\\.([0-9]+))?(\\.([0-9]+))?")
def recupera_versao():
  try:
    p = open(caminho)
    s = p.read()
    match = reg.match(s)
    versoes = (int(match.groups()[0]),)
    for i in range(2, len(match.groups()),2):
      versoes = versoes + (int(match.groups()[i]),)
    return versoes
  except Exception as ex:
    raise ErroVersao("Impossível ler versão do sistema operacional") from ex

def carrega_lib(versao):
  print("Carregando bibliotecas da versão " + str(versao))
  # carrega bibliotecas de uma determinada versão

def carrega_bibliotecas():
  try:
    versao = recupera_versao()
    if versao[0]>=18:
      carrega_lib(3)
    elif versao[0]>=16 and versao[1]<18:
      carrega_lib(2)
    else:
      raise RuntimeError("Versão não suportada")
  except ErroVersao as ex:
      raise RuntimeError("Impossível determinar a versão") from ex


O código a seguir introduz um erro deliberado no caminho do arquivo de versão a ser lido:

In [ ]:
caminho = "/caminho_inexistente/"
carrega_bibliotecas()

RuntimeError: Impossível determinar a versão

Observe como a exceção do tipo ```ErroVersao``` é tratada especificamente dentro da função ```carrega_bibliotecas```.

Note também a sintaxe do bloco ```except``` com a plavra-chave ```as``` seguida de um nome de variável.
Este nome torna-se uma variável definida *dentro* do bloco except. Ele permite acessar diretamente o objeto lançado e recuperar as informações nele armazenadas.

Objetos de classes geradas pelo programador podem naturalmente conter quaisquer campos pertinentes. A classe ```BaseException``` no entanto, admite no seu construtor um número variável de parâmetros que podem ser recuperados pelo campo ```args```:



In [ ]:
caminho = "/caminho_inexistente/"
try:
  carrega_bibliotecas()
except RuntimeError as erro:
  print("Um Erro aconteceu:")
  print(erro.args)

Um Erro aconteceu:
('Impossível determinar a versão', 1, 2, 3)


Um exemplo mais abstrato mostrando múltiplos parâmetros.

In [ ]:
 def func():
   raise Exception("parâmetro 1", "parâmetro 2", "parâmetro 3")

try:
  func()
except Exception as erro:
  for x in erro.args:
    print(x)

parâmetro 1
parâmetro 2
parâmetro 3


Note também o recurso de se "encadear" exceções".

Como a seleção do bloco que vai tratar a exceção se dá pelo *tipo*, às vezes é útil re-envelopar a exceção lançada de forma que o tipo transmita de forma mais clara o que aconteceu.

No exemplo ```recupera_versao()```, o cõdigo pode lançar exeções do tipo ```IOError``` caso o arquivo não possa ser lido, ```AttributeError```, ```IndexError``` ou ```ValueError``` caso o seu conteúdo não possa ser interpretado.

Pode ser difícil para um módulo distante da implementação ```recupera_versao()``` entender o que exatamente aconteceu!
Ao enveloparmos a exceção original dentro de uma nova exceção do tipo ```ErroVersao```, fica mais claro que, independentemente da razão específica, o que aconteceu foi um ```ErroVersao```.
Os dados da exceção original pode ser preservada.

A sintaxe é:
```
raise <nova exceção> from <objeto da exceção original>
```
Por exemplo:

In [ ]:
 def func():
   raise Exception("Exceção 1")

try:
  func()
except Exception as erro:
  raise Exception("Exceção 2") from erro

Exception: ignored

Blocos de tratamento de exceção admitem uma cláusula ```else``` a ser executada caso *nenhuma* exceção ocorra:

In [ ]:
s = [1,2,3,4]

for i in range(5):
  try:
    print(s[i])
  except:
    print("ops!")
  else:
    print("deu certo!")


1
deu certo!
2
deu certo!
3
deu certo!
4
deu certo!
ops!


Além disso, é possível declarar um bloco que é *sempre* executado, ocorrendo ou não uma exceção por meio da palavra-chave ```finally```.

Observe como no exemplo abaixo, a mensagem "saindo de f2" é impressa, mesmo com o lançamento de uma exceção em ```f1```.

In [ ]:
def f1():
  print("Entrando em f1")
  raise Exception()

def f2():
  try:
    print("Entrando em f2")
    f1()
    print("f2 chamou f1")
  finally:
    print("Saindo de f2")

def f3():
  print("Entrando em f3")
  f2()
  print("Saindo de f3")

f3()


Entrando em f3
Entrando em f2
Entrando em f1
Saindo de f2


Exception: ignored